# SHE: Sentiment Hashtag Embedding Through Multitask learning

In [1]:
import numpy as np
import gc
import pickle
import gensim

In [2]:
stopwords = ['rt','amp','url','sir','day','title','shri','crore','time',"a", "about","above", "across", "after", "afterwards", "again", "all", "almost", "alone", "along", "already", "also","although","always","am","among", "amongst", "amoungst", "amount",  "an", "and", "another", "any","anyhow","anyone","anything","anyway", "anywhere", "are", "around", "as",  "at", "back","be","became", "because","become","becomes", "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "bill", "both", "bottom","but", "by", "call", "can", "cannot", "cant", "co", "con", "could", "couldnt", "cry", "de", "describe", "detail", "do", "done", "down", "due", "during", "each", "eg", "eight", "either", "eleven","else", "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "fifteen", "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly", "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "ie", "if", "in", "inc", "indeed", "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much", "must", "my", "myself", "name", "namely", "neither", "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own","part", "per", "perhaps", "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "serious", "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take", "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "thickv", "thin", "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too", "top", "toward", "towards", "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we", "well", "were", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves", "the"]

### Load pre-trained embedding

In [3]:
model = gensim.models.KeyedVectors.load_word2vec_format('emb/42K_meta_mve.emb', binary=False) 

In [4]:
idx=1
words = []
word2idx = {}
vectors=[]
vect_idx=[]

vectors.append(np.asarray([0]*model.vector_size))
with open('emb/42K_meta_mve.emb', 'rb') as f:
    for i,l in enumerate(f):
        if i > 0:
            line = l.decode().split()
            if line[1] != '-nan':
                word = line[0]
                words.append(word)
                word2idx[word] = idx
                vect = line[1:]
                vectors.append(np.asarray(vect))
                vect_idx.append(idx)
                idx += 1
                
print(len(vectors), len(word2idx), len(vectors[0]))

37090 37089 100


In [5]:
emb=len(vectors[0])
vectors=np.asarray(vectors)
vect_idx=np.array(vect_idx)
tss=int(len(word2idx) * 0.8)   #Train-test data split 80-20 for autoencoder
train, test = vectors[:tss][:], vectors[tss:][:]
train_idx, test_idx = vect_idx[:tss][:], vect_idx[tss:][:]

### Load available sentiment lexicon

In [6]:
sent=dict()
f=open('data/Tweets_sentilexicon_final_auto', 'r')
lines=f.readlines()
for l in lines:
    word=l.lower().strip().split('\t')
    if word[1] not in sent:
        sent[word[1]]=[]
    sent[word[1]].append(word[0])
f.close()


In [7]:
def gen_index(wlist,word2idx):
    ip=[]
    for i in wlist:
        try:
            idx=word2idx[i]
            ip.append(idx)
        except:
            pass
    return ip

def split_list(alist, wanted_parts=1):
    length = len(alist)
    return [ alist[i*length // wanted_parts: (i+1)*length // wanted_parts] 
             for i in range(wanted_parts) ]


In [8]:
folds=dict()

positive = gen_index(sent['positive'],word2idx)
negative = gen_index(sent['negative'],word2idx)
neutral = gen_index(sent['neutral'],word2idx)

In [9]:
from random import shuffle

print("Data preparation for fold ")

fold=[]
for idx in sent['positive']:
    fold.append(idx)
shuffle(fold)
folds['positive']=split_list(fold, wanted_parts=10)

fold=[]
for idx in sent['negative']:
    fold.append(idx)
shuffle(fold)
folds['negative']=split_list(fold, wanted_parts=10)

fold=[]
for idx in sent['neutral']:
    fold.append(idx)
shuffle(fold)
folds['neutral']=split_list(fold, wanted_parts=10)

Data preparation for fold 


### Building SHE model

In [10]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional,SimpleRNN,GRU,CuDNNGRU,Reshape
from keras.layers import Conv1D, GlobalMaxPooling1D, Activation, Flatten, UpSampling1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import plot_model
import tensorflow as tf
from keras.models import model_from_json
import pickle
from keras import backend as K

Using TensorFlow backend.


In [11]:
for fi in range(1):   #Number of folds. Here I kept 1 as for demonstration
    testing=[]
    for sample in folds['positive'][fi]:
        try:
            idx=word2idx[sample]
            testing.append([idx,[1,0,0]])
        except:
            pass
    
    for sample in folds['negative'][fi]:
        try:
            idx=word2idx[sample]
            testing.append([idx,[0,1,0]])
        except:
            pass
    
    for sample in folds['neutral'][fi]:
        try:
            idx=word2idx[sample]
            testing.append([idx,[0,0,1]])
        except:
            pass
    
    #testing+=folds['neutral'][fi]
    training=[]
    for fj in range(10):
        if fj != fi:
            for sample in folds['positive'][fj]:
                try:
                    idx=word2idx[sample]
                    training.append([idx,[1,0,0]])
                except:
                    pass
            
            for sample in folds['negative'][fj]:
                try:
                    idx=word2idx[sample]
                    training.append([idx,[0,1,0]])
                except:
                    pass
            
            for sample in folds['neutral'][fj]:
                try:
                    idx=word2idx[sample]
                    training.append([idx,[0,0,1]])
                except:
                    pass
            
            #training+=folds['neutral'][fj]
   
    print( "Training fold begin",fi)

    training_labels=[]
    testing_labels=[]

    training_idx=[]
    testing_idx=[]

    training_AE=[]
    testing_AE=[]   # = sent_vec[:TSS][:], sent_vec[TSS:][:]
    for i in training:
        idx=i[0]
        training_labels.append(i[1])
        training_AE.append(vectors[idx])
        training_idx.append(idx)

    training_AE=np.array(training_AE)
    training_labels=np.array(training_labels)
    training_idx=np.array(training_idx)

    for i in testing:
        idx=i[0]
        testing_labels.append(i[1])
        testing_AE.append(vectors[idx])
        testing_idx.append(idx)

    testing_AE=np.array(testing_AE)
    testing_labels=np.array(testing_labels)
    testing_idx=np.array(testing_idx)
    #for i in range(10):
    input_node=Input(shape=(emb,))
    #encode=Embedding(len(word2idx), emb, weights=[vectors], input_length=1,trainable=True)(input_node)
    encode=Reshape((1,emb))(input_node)
    encode=Conv1D(emb,
                     3,
                     padding='same',
                     activation='relu',
                     strides=1)(encode)
    encode=MaxPooling1D(pool_size=1)(encode)
    decoder=Dropout(0.2)(encode)
    decoder=Conv1D(64,
                     3,
                     padding='same',
                     activation='relu',
                     strides=1)(decoder)
    decoder=GlobalMaxPooling1D()(decoder)
    decode=Dense(emb,activation='tanh',name="dense_1")(decoder)
    
    classifier=Conv1D(64,
                     3,
                     padding='same',
                     activation='relu',
                     strides=1)(encode)
    classifier=Dropout(0.2)(classifier)
    classifier=GlobalMaxPooling1D()(classifier)
    senti_class=Dense(3,activation='softmax',name="dense_2")(classifier)


    encoder = Model(input_node, encode)
    
    autoencoder=Model(input_node,decode)
    autoencoder.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    autoencoder.summary()

    
    combine=Model(input_node,[decode,senti_class])

    losses = {
        "dense_1": "mean_squared_error",
        "dense_2": "categorical_crossentropy",
    }
    lossWeights = {"dense_1": 1.0, "dense_2": 1.0}  # Weightage for optimizing the error loss
    combine.compile(optimizer='adam', loss=losses, loss_weights=lossWeights,
        metrics=["accuracy"])
    combine.summary()
    for learn in range(5):
        autoencoder.fit(train, train,
                    epochs=20,
                    batch_size=128,
                    shuffle=True,
                    validation_data=(test, test))
                    
        combine.fit(training_AE, [training_AE, training_labels],
                    epochs=20,
                    batch_size=128,
                    shuffle=True,
                    validation_data=(testing_AE, [testing_AE, testing_labels]))
    
    model_json = combine.to_json()
    with open("model/senti_autoencoder_combine_"+str(fi)+".json", "w") as json_file:
        json_file.write(model_json)
    combine.save_weights("model/senti_autoencoder_combine_"+str(fi)+".h5")

    model_json = encoder.to_json()
    with open("model/3class_encoder_"+str(fi)+".json", "w") as json_file:
        json_file.write(model_json)
    encoder.save_weights("model/3class_encoder_"+str(fi)+".h5")
    
        
    f2=open('model/SHE_encoder_out_'+str(fi)+'emb','w')
    f2.write(str(len(word2idx))+' '+str(emb)+'\n')
    for key in word2idx:
        f2.write(key.replace(' ','')+" ")
        w2v=model[key]
        result=encoder.predict(np.asarray([w2v]),verbose=0)
        for i in list(result[0][0]):
            f2.write(str(i)+" ")
        f2.write("\n")
    f2.close()

    
    del input_node
    del encode
    del classifier
    del senti_class
    del autoencoder
    del combine
    gc.collect()   
    K.clear_session()


Training fold begin 0
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 1, 100)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1, 100)            30100     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 1, 100)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 100)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1, 64)             19264     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 6

Epoch 4/20
7740/7740 [==============================] - 2s 196us/step - loss: 1.0608 - dense_1_loss: 0.0027 - dense_2_loss: 1.0585 - dense_1_accuracy: 0.3757 - dense_2_accuracy: 0.4437 - val_loss: 1.0463 - val_dense_1_loss: 0.0019 - val_dense_2_loss: 1.0327 - val_dense_1_accuracy: 0.4712 - val_dense_2_accuracy: 0.4447
Epoch 5/20
7740/7740 [==============================] - 1s 187us/step - loss: 1.0565 - dense_1_loss: 0.0027 - dense_2_loss: 1.0540 - dense_1_accuracy: 0.3689 - dense_2_accuracy: 0.4519 - val_loss: 1.0439 - val_dense_1_loss: 0.0019 - val_dense_2_loss: 1.0279 - val_dense_1_accuracy: 0.4760 - val_dense_2_accuracy: 0.4471
Epoch 6/20
7740/7740 [==============================] - 2s 196us/step - loss: 1.0512 - dense_1_loss: 0.0027 - dense_2_loss: 1.0481 - dense_1_accuracy: 0.3762 - dense_2_accuracy: 0.4547 - val_loss: 1.0430 - val_dense_1_loss: 0.0020 - val_dense_2_loss: 1.0236 - val_dense_1_accuracy: 0.4471 - val_dense_2_accuracy: 0.4507
Epoch 7/20
7740/7740 [==================

29671/29671 [==============================] - 6s 200us/step - loss: 0.0025 - accuracy: 0.3951 - val_loss: 0.0017 - val_accuracy: 0.5202
Epoch 20/20
29671/29671 [==============================] - 6s 200us/step - loss: 0.0025 - accuracy: 0.4002 - val_loss: 0.0017 - val_accuracy: 0.5297
Train on 7740 samples, validate on 832 samples
Epoch 1/20
7740/7740 [==============================] - 1s 191us/step - loss: 1.0870 - dense_1_loss: 0.0026 - dense_2_loss: 1.0845 - dense_1_accuracy: 0.3857 - dense_2_accuracy: 0.4300 - val_loss: 1.0614 - val_dense_1_loss: 0.0019 - val_dense_2_loss: 1.0497 - val_dense_1_accuracy: 0.4627 - val_dense_2_accuracy: 0.4567
Epoch 2/20
7740/7740 [==============================] - 1s 193us/step - loss: 1.0416 - dense_1_loss: 0.0025 - dense_2_loss: 1.0392 - dense_1_accuracy: 0.3885 - dense_2_accuracy: 0.4602 - val_loss: 1.0475 - val_dense_1_loss: 0.0018 - val_dense_2_loss: 1.0344 - val_dense_1_accuracy: 0.4820 - val_dense_2_accuracy: 0.4627
Epoch 3/20
7740/7740 [=====

29671/29671 [==============================] - 6s 199us/step - loss: 0.0024 - accuracy: 0.3985 - val_loss: 0.0016 - val_accuracy: 0.5299
Epoch 11/20
29671/29671 [==============================] - 6s 199us/step - loss: 0.0024 - accuracy: 0.3940 - val_loss: 0.0016 - val_accuracy: 0.5307
Epoch 12/20
29671/29671 [==============================] - 6s 197us/step - loss: 0.0024 - accuracy: 0.4007 - val_loss: 0.0016 - val_accuracy: 0.5359
Epoch 13/20
29671/29671 [==============================] - 6s 201us/step - loss: 0.0024 - accuracy: 0.3973 - val_loss: 0.0016 - val_accuracy: 0.5282
Epoch 14/20
29671/29671 [==============================] - 6s 199us/step - loss: 0.0024 - accuracy: 0.3967 - val_loss: 0.0016 - val_accuracy: 0.5297
Epoch 15/20
29671/29671 [==============================] - 6s 204us/step - loss: 0.0024 - accuracy: 0.4006 - val_loss: 0.0016 - val_accuracy: 0.5338
Epoch 16/20
29671/29671 [==============================] - 6s 200us/step - loss: 0.0024 - accuracy: 0.4009 - val_loss:

29671/29671 [==============================] - 6s 218us/step - loss: 0.0026 - accuracy: 0.3855 - val_loss: 0.0016 - val_accuracy: 0.5251
Epoch 2/20
29671/29671 [==============================] - 7s 223us/step - loss: 0.0025 - accuracy: 0.3992 - val_loss: 0.0016 - val_accuracy: 0.5282
Epoch 3/20
29671/29671 [==============================] - 7s 226us/step - loss: 0.0024 - accuracy: 0.3980 - val_loss: 0.0016 - val_accuracy: 0.5268
Epoch 4/20
29671/29671 [==============================] - 8s 266us/step - loss: 0.0024 - accuracy: 0.4014 - val_loss: 0.0016 - val_accuracy: 0.5303
Epoch 5/20
29671/29671 [==============================] - 7s 242us/step - loss: 0.0024 - accuracy: 0.3960 - val_loss: 0.0016 - val_accuracy: 0.5326
Epoch 6/20
29671/29671 [==============================] - 6s 215us/step - loss: 0.0024 - accuracy: 0.4004 - val_loss: 0.0016 - val_accuracy: 0.5321
Epoch 7/20
29671/29671 [==============================] - 6s 218us/step - loss: 0.0024 - accuracy: 0.3998 - val_loss: 0.001

7740/7740 [==============================] - 2s 221us/step - loss: 0.7734 - dense_1_loss: 0.0025 - dense_2_loss: 0.7711 - dense_1_accuracy: 0.3764 - dense_2_accuracy: 0.6553 - val_loss: 1.2030 - val_dense_1_loss: 0.0019 - val_dense_2_loss: 1.1749 - val_dense_1_accuracy: 0.4471 - val_dense_2_accuracy: 0.4243
Epoch 18/20
7740/7740 [==============================] - 2s 206us/step - loss: 0.7663 - dense_1_loss: 0.0025 - dense_2_loss: 0.7637 - dense_1_accuracy: 0.3801 - dense_2_accuracy: 0.6567 - val_loss: 1.2190 - val_dense_1_loss: 0.0019 - val_dense_2_loss: 1.2007 - val_dense_1_accuracy: 0.4555 - val_dense_2_accuracy: 0.4183
Epoch 19/20
7740/7740 [==============================] - 2s 216us/step - loss: 0.7453 - dense_1_loss: 0.0025 - dense_2_loss: 0.7433 - dense_1_accuracy: 0.3748 - dense_2_accuracy: 0.6778 - val_loss: 1.2339 - val_dense_1_loss: 0.0019 - val_dense_2_loss: 1.2034 - val_dense_1_accuracy: 0.4423 - val_dense_2_accuracy: 0.4195
Epoch 20/20
7740/7740 [==========================

7740/7740 [==============================] - 1s 190us/step - loss: 0.8043 - dense_1_loss: 0.0025 - dense_2_loss: 0.8020 - dense_1_accuracy: 0.3879 - dense_2_accuracy: 0.6443 - val_loss: 1.1799 - val_dense_1_loss: 0.0018 - val_dense_2_loss: 1.1698 - val_dense_1_accuracy: 0.4507 - val_dense_2_accuracy: 0.4075
Epoch 14/20
7740/7740 [==============================] - 2s 198us/step - loss: 0.8009 - dense_1_loss: 0.0025 - dense_2_loss: 0.7981 - dense_1_accuracy: 0.3798 - dense_2_accuracy: 0.6433 - val_loss: 1.1929 - val_dense_1_loss: 0.0018 - val_dense_2_loss: 1.1836 - val_dense_1_accuracy: 0.4651 - val_dense_2_accuracy: 0.4099
Epoch 15/20
7740/7740 [==============================] - 2s 195us/step - loss: 0.7811 - dense_1_loss: 0.0025 - dense_2_loss: 0.7800 - dense_1_accuracy: 0.3839 - dense_2_accuracy: 0.6547 - val_loss: 1.2014 - val_dense_1_loss: 0.0018 - val_dense_2_loss: 1.1836 - val_dense_1_accuracy: 0.4543 - val_dense_2_accuracy: 0.4219
Epoch 16/20
7740/7740 [==========================

### Save the SHE trained embedding from the best performaning fold.

In [12]:
# f2=open('emb/SHE_encoder_out.emb','w')
# f2.write(str(len(word2idx))+' '+str(emb)+'\n')
# for key in word2idx:
#     f2.write(key.replace(' ','')+" ")
#     w2v=model[key]
#     result=encoder.predict(np.asarray([w2v]),verbose=0)
#     for i in list(result[0][0]):
#         f2.write(str(i)+" ")
#     f2.write("\n")
# f2.close()